In [1]:
import json

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None

In [2]:
SRC_TRAIN = "../../../data/src/train.csv"
TRAIN = "../../../data/train.csv"
TEST = "../../../data/test.csv"
CONFIG = "../../../data/conf.json"

# Чтение данных

In [3]:
df = pd.read_csv(SRC_TRAIN)

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Избавимся от столбцов, которые для выбранного решения "кажутся" бесполезными

In [5]:
df = df.drop("PassengerId", axis=1)
df = df.drop("Name", axis=1)
df = df.drop("Ticket", axis=1)
df = df.drop("Cabin", axis=1)

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Случайное разделение исходных данных на train/test датасеты в пропорции 0,7 / 0,3

In [7]:
train, test = train_test_split(df, test_size=0.3)

Сохранение полученных данных для использования в обучении NeoML

In [8]:
train.to_csv(TRAIN, sep="|")
test.to_csv(TEST, sep="|")

In [9]:
train.count()

Survived    623
Pclass      623
Sex         623
Age         486
SibSp       623
Parch       623
Fare        623
Embarked    621
dtype: int64

Неодинаковое кол-во значений в некоторых столбцах говорит о том, что данные нужно 'полечить'

In [10]:
datasets = [train, test]

# Подготовка данных

## Age

Для исправления пропусков в возрасте будем выбирать случайное значение из интервала (mean - sigma; mean + sigma)

In [11]:
age_mean = train["Age"].mean()
age_std = train["Age"].std()

print("age-mean='{0}' age-std='{1}'".format(age_mean, age_std))

age-mean='29.71656378600823' age-std='14.578734474903417'


In [12]:
for ds in datasets:
    age_null_count = ds["Age"].isnull().sum()
    rand_age = np.random.randint(age_mean - age_std, age_mean + age_std, size=age_null_count)
    age_slice = ds["Age"].copy()
    age_slice[ np.isnan(age_slice) ] = rand_age
    ds["Age"] = age_slice
    ds["Age"] = ds["Age"].astype(int)

## Embarked

Будем считать, что отсутствующее значение равно S (Southampton)

In [13]:
for ds in datasets:
    ds["Embarked"] = ds["Embarked"].fillna("S")

In [14]:
train.count()

Survived    623
Pclass      623
Sex         623
Age         623
SibSp       623
Parch       623
Fare        623
Embarked    623
dtype: int64

## Encodes

Строковые данные пола и места посадки кодируются с помощью LabelEncoder

In [15]:
sex_encoder = LabelEncoder()
sex_encoder.fit(train["Sex"])

train["Sex"] = sex_encoder.transform(train["Sex"])
test["Sex"] = sex_encoder.transform(test["Sex"])

In [16]:
embarked_encode = LabelEncoder()
embarked_encode.fit(train["Embarked"])

train["Embarked"] = embarked_encode.transform(train["Embarked"])
test["Embarked"] = embarked_encode.transform(test["Embarked"])

## X/Y

Из исходных данных подготовим сэмплы для обучения классификатора

In [17]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]

X_test = test.drop("Survived", axis=1)
Y_test = test["Survived"]

## Scale

Вектора признаков необъодимо скалировать, что бы значения некоторых фич "не давили" своим весом

In [18]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Обучение

## Train

Для классификатора выберем модель на основе SVM

In [19]:
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)

SVC(random_state=0)

## Test

In [20]:
Y_train_predict = classifier.predict(X_train)
acc = accuracy_score(Y_train, Y_train_predict)

print("Train Acc = {:.2f}".format(acc * 100))

Train Acc = 83.79


In [21]:
Y_test_predict = classifier.predict(X_test)
acc = accuracy_score(Y_test, Y_test_predict)

print("Test Acc = {:.2f}".format(acc * 100))

Test Acc = 84.33


Запомним значения точности, чтобы сверить с решением на NeoML

# Сохранение конфига

Для воспроизведения функционала подготовки данных сохраняем параметры в json-конфиг

In [22]:
# Количество полей + 1 (столбец с индексом)
fields_count = len(train.count()) + 1

config = {
    'expected-fields': fields_count,
    'min-age': int(age_mean - age_std),
    'max-age': int(age_mean + age_std),
    'sex-labels': list(sex_encoder.classes_),
    'embarked-labels': list(embarked_encode.classes_),
    'scaler-mean': list(scaler.mean_),
    'scaler-std': list(scaler.scale_)
}

with open(CONFIG, 'w') as fp:
    json.dump(config, fp)

# NeoML incoming

Итак, после исследования имеется в наличии:
  - процедура подготовки данных и конфиг, описывающий параметры;
  - модель, решающая задачу.

Необходимо воспроизвести решение с использованием NeoML.
Это позволит создать SDK, которое можно интегрировать в произвольное приложение

Что ж, приступим...